In [ ]:
# Install deepinv (skip if already installed)
%pip install deepinv

<!-- MathJax macro definitions inserted automatically -->
$$
\newcommand{\forw}[1]{{A\left({#1}\right)}}
\newcommand{\noise}[1]{{N\left({#1}\right)}}
\newcommand{\inverse}[1]{{R\left({#1}\right)}}
\newcommand{\inversef}[2]{{R\left({#1},{#2}\right)}}
\newcommand{\inversename}{R}
\newcommand{\reg}[1]{{g_\sigma\left({#1}\right)}}
\newcommand{\regname}{g_\sigma}
\newcommand{\sensor}[1]{{\eta\left({#1}\right)}}
\newcommand{\datafid}[2]{{f\left({#1},{#2}\right)}}
\newcommand{\datafidname}{f}
\newcommand{\distance}[2]{{d\left({#1},{#2}\right)}}
\newcommand{\distancename}{d}
\newcommand{\denoiser}[2]{{\operatorname{D}_{{#2}}\left({#1}\right)}}
\newcommand{\denoisername}{\operatorname{D}_{\sigma}}
\newcommand{\xset}{\mathcal{X}}
\newcommand{\yset}{\mathcal{Y}}
\newcommand{\group}{\mathcal{G}}
\newcommand{\metric}[2]{{d\left({#1},{#2}\right)}}
\newcommand{\loss}[1]{{\mathcal\left({#1}\right)}}
\newcommand{\conj}[1]{{\overline{#1}^{\top}}}
$$


# 5 minute quickstart tutorial

Follow this example to get started with DeepInverse in under 5 minutes.

**Contents**

1. [Install](#install)_
2. [Physics](#physics)_
3. [Models](#models)_
4. [Datasets](#datasets)_
5. [What's next](#what-s-next)_

## 1. Install

First, install and import the latest stable release of `deepinv`:

> **Code**: bash
>
>
> pip install deepinv
>

We then get the device (CPU in the case of this example).



In [ ]:
import deepinv as dinv
import torch

device = dinv.utils.get_freer_gpu() if torch.cuda.is_available() else "cpu"

## 2. Physics



In DeepInverse, `x` are images:



In [ ]:
x = dinv.utils.load_example("butterfly.png", device=device)

Images are tensors of shape `B, C, ...` where `B` is batch size, `C` are channels and `...` are spatial dimensions:


In [ ]:
print(x.shape)

Imaging forward operators are called `physics` and simulate
measurements `y` from `x`.



In [ ]:
physics = dinv.physics.Inpainting(x.shape[1:], mask=0.3, device=device)

y = physics(x)

DeepInverse implements
many different types of physics across various imaging modalities.
Physics also possess noise models such as Gaussian or Poisson noise.

> **Hint**
>
> Do you get a matplotlib LaTeX error when plotting? Disable LaTeX using `dinv.utils.disable_tex()`
>


In [ ]:
physics.noise_model = dinv.physics.GaussianNoise(sigma=0.1)

y = physics(x)

dinv.utils.plot({"GT": x, "Noisy Inpainting \nMeasurement": y})

Many physics also take
physics parameters such as `mask`, `filter`, `sigma` etc.:


In [ ]:
# Blur with Gaussian filter parameter
filter = dinv.physics.blur.gaussian_blur((5, 5))

physics = dinv.physics.BlurFFT(x.shape[1:], filter=filter, device=device)

# Simulate measurements
y = physics(x)

You can easily use your own params by passing these into the `physics`,
or you can use a `generator` to generate random params:


In [ ]:
# Blur kernel random generator
physics_generator = dinv.physics.generator.MotionBlurGenerator(
    psf_size=(31, 31), num_channels=3, device=device
)

# Generate a dict of random params {"filter": ...}
params = physics_generator.step()

# Update physics during forward call
y2 = physics(x, **params)

dinv.utils.plot(
    {
        "GT": x,
        "Blurred...": y,
        "... with Gaussian kernel": filter,
        "Blurred ...": y2,
        "...with motion kernel": params["filter"],
    }
)

Physics are powerful objects and have many methods, for example a
pseudo-inverse:



In [ ]:
# You can also update params like so
physics.update(filter=filter.to(device))

x_pinv = physics.A_dagger(y)

As it is well-known in the field of inverse problems, the pseudo-inverse can give good results
if the problem is noiseless, but it completely fails in the presence of noise - this is why we need reconstructors!


In [ ]:
physics.noise_model = dinv.physics.GaussianNoise(sigma=0.1)

y = physics(x)

x_pinv_noise = physics.A_dagger(y)

dinv.utils.plot(
    {"Pseudoinv \nw/o noise": x_pinv, "Pseudoinv \nwith noise": x_pinv_noise}
)

> **Tip**
>
>
> Want to use DeepInverse with your own physics operator? Check out [`basics/demo_custom_physics.py`](https://deepinv.github.io/deepinv/auto_examples/basics/demo_custom_physics.html#sphx-glr-auto-examples-basics-demo-custom-physics-py) for a tutorial!
>
>


## 3. Models

In DeepInverse, a `model` is a reconstruction algorithm that
**reconstructs** images from `y` and knowledge of `physics`.

> **Tip**
>
> Many models, such as [Reconstruct Anything Model](https://deepinv.github.io/deepinv/api/stubs/deepinv.models.RAM.html), are pretrained reconstructors and can
> be used out of the box. See [`basics/demo_pretrained_model.py`](https://deepinv.github.io/deepinv/auto_examples/basics/demo_pretrained_model.html#sphx-glr-auto-examples-basics-demo-pretrained-model-py) for a full example.
>
>


In [ ]:
model = dinv.models.RAM(pretrained=True, device=device)

x_hat = model(y, physics)

Plot the image `x`, the measurement `y` and the reconstructed image
`x_hat` and compute metrics:



In [ ]:
metric = dinv.metric.PSNR()

psnr_y = metric(y, x).item()
psnr_x_hat = metric(x_hat, x).item()

dinv.utils.plot(
    {
        "Ground Truth": x,
        "Measurement": y,
        "Reconstruction": x_hat,
    },
    subtitles=["PSNR:", f"{psnr_y:.2f} dB", f"{psnr_x_hat:.2f} dB"],
)

Some models are only denoisers that **denoise**
images from `y` and `sigma`, which can be used to build many
model-based reconstruction algorithms.



In [ ]:
denoiser = dinv.models.DRUNet(device=device)

x_denoised = denoiser(y, sigma=0.1)

model = dinv.optim.DPIR(sigma=0.1, denoiser=denoiser, device=device)

x_hat = model(y, physics)

dinv.utils.plot(
    {
        "Ground Truth": x,
        "Measurement": y,
        "Denoised": x_denoised,
        "Reconstruction": x_hat,
    },
    subtitles=[
        "PSNR:",
        f"{dinv.metric.PSNR()(y, x).item():.2f} dB",
        f"{dinv.metric.PSNR()(x_denoised, x).item():.2f} dB",
        f"{dinv.metric.PSNR()(x_hat, x).item():.2f} dB",
    ],
)

DeepInverse covers
many frameworks of reconstruction algorithms
including deep model architectures, iterative algorithms, sampling algorithms
(e.g. diffusion models), and unfolded models.



In [ ]:
# Reconstruct Anything Model foundation model
model = dinv.models.RAM(pretrained=True, device=device)

> **Tip**
>
>
> Want to use DeepInverse with your own network? Just inherit from the reconstructor base class [`deepinv.models.Reconstructor`](https://deepinv.github.io/deepinv/api/stubs/deepinv.models.Reconstructor.html)!
>
>


## 4. Datasets

You can use DeepInverse with dataset, for testing or training. First,
define a ground-truth dataset. We implement wrappers for
many popular imaging datasets across domains including natural images,
medical imaging, satellite imaging, etc.

> **Tip**
>
> It's easy to use your own dataset with DeepInverse. See [`basics/demo_custom_dataset.py`](https://deepinv.github.io/deepinv/auto_examples/basics/demo_custom_dataset.html#sphx-glr-auto-examples-basics-demo-custom-dataset-py) for a tutorial.
>
>


In [ ]:
dataset = dinv.datasets.SimpleFastMRISliceDataset(
    "data", anatomy="brain", download=True
)

Datasets return either `x`, tuples `x, y` or `x, y, params` of images,
measurements, and optional physics parameters. Given a ground-truth
dataset, you can simulate a dataset with random physics:



In [ ]:
physics = dinv.physics.MRI(device=device)

physics_generator = dinv.physics.generator.RandomMaskGenerator(
    (320, 320), device=device
)

path = dinv.datasets.generate_dataset(
    dataset,
    physics,
    save_dir="data",
    physics_generator=physics_generator,
    device=device,
)

dataset = dinv.datasets.HDF5Dataset(path, load_physics_generator_params=True)

You can use this dataset to test or train a model:



In [ ]:
import torch

dinv.test(
    model,
    torch.utils.data.DataLoader(dataset),
    physics,
    plot_images=True,
    device=device,
)

> **Tip**
>
>
> Want to use DeepInverse with your own dataset? Check out [`basics/demo_custom_dataset.py`](https://deepinv.github.io/deepinv/auto_examples/basics/demo_custom_dataset.html#sphx-glr-auto-examples-basics-demo-custom-dataset-py) for a tutorial!
>
>


🎉 Well done, you now know how to use DeepInverse!

## What's next?

-  Try more basic examples, including
   [how to inference a pretrained model](https://deepinv.github.io/deepinv/auto_examples/basics/demo_pretrained_model.html#sphx-glr-auto-examples-basics-demo-pretrained-model-py),
   [how to use your own dataset](https://deepinv.github.io/deepinv/auto_examples/basics/demo_custom_dataset.html#sphx-glr-auto-examples-basics-demo-custom-dataset-py), or
   [how to use your custom physics operator](https://deepinv.github.io/deepinv/auto_examples/basics/demo_custom_physics.html#sphx-glr-auto-examples-basics-demo-custom-physics-py).
-  Dive deeper into our full library of examples.
-  Read the User Guide for further details on the
   concepts introduced here.
-  Want help?
   [Open an issue](https://github.com/deepinv/deepinv/issues) ask
   a message on our [Discord](https://discord.gg/qBqY5jKw3p) or
   get in touch with our
   [MAINTAINERS](https://github.com/deepinv/deepinv/blob/main/MAINTAINERS.md).

